In [1]:
import torch.nn as nn

In [7]:
class VGG(nn.Module):
    def __init__(self, features, num_classes=2, init_weights=True):
        super(VGG, self).__init__()

        self.features = features  # convolution

        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))

        self.classifier = nn.Sequential(
            nn.Linear(512 * 4 * 4, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )  # FC layer

        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)  # Convolution
        print(f"첫번째 x.shape : {x.shape}")
        x = self.avgpool(x)  # avgpool
        print(f"avgP x.shape : {x.shape}")
        x = x.view(x.size(0), -1)
        print(f"View x.shape : {x.shape}")
        x = self.classifier(x)  # FC layer
        print(f"마지막 x.shape : {x.shape}")
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(
                    m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


In [8]:
def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3

    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v

    return nn.Sequential(*layers)


In [9]:
cfg = {
    # 8 + 3 =11 == vgg11
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    # 10 + 3 = vgg 13
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    # 13 + 3 = vgg 16
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    # 16 +3 =vgg 19
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
    'custom': [64, 64, 64, 'M', 128, 128, 128, 'M', 256, 256, 256, 'M']
}

In [10]:
conv = make_layers(cfg['D'], batch_norm=True)

In [11]:
CNN = VGG(make_layers(cfg['D']), num_classes=10, init_weights=True)